In [59]:
from data import Dataset

%load_ext autoreload
%autoreload 2

import torch

from metrics import NDCG_binary_at_k_batch

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [60]:
class dotdict(dict):
        """dot.notation access to dictionary attributes"""
        __getattr__ = dict.get
        __setattr__ = dict.__setitem__
        __delattr__ = dict.__delitem__

In [61]:
args = {}

args = dotdict(args)
args['data'] = 'foursquare' #'gowalla'

args['model'] = 'BYOL_reco_model' #SimCLR_reco_model, MultiVAE, BYOL_reco_model

args['n_epoches'] = 100

args['n_epoches_dec'] = 50

args['lrdec'] = 1e-3

args['lrenc'] = 1e-3

args['train_batch_size'] = 500

args['val_batch_size'] = 200

args['n_val_samples'] = 1

args['annealing'] = True

args.total_anneal_steps = 50

args.anneal_cap = 1.

args.print_info_ = 5

args.metric = NDCG_binary_at_k_batch

args.device = 'cpu'

args.n_views = 2

args.temperature = .07

args.dropout_rate = 0.83

args.criterion_dec = torch.nn.CrossEntropyLoss()

args.m = 0.995

In [66]:
dataset = Dataset(args)
# pdb.set_trace()
layers = [100, 600, dataset.n_items]
args.z_dim = layers[0]
args.l2_coeff = 0.

In [71]:
from other_models import *
from pythae.models import VAMPConfig, VAEConfig, VAE_LinNF_Config, VAE_IAF_Config
from pythae.models.nn import BaseEncoder, BaseDecoder
from pythae.models.base.base_utils import ModelOutput
from models import make_linear_network


class Encoder(BaseEncoder):
    def __init__(self, dims):
        BaseDecoder.__init__(self)
        self.layers = make_linear_network(
            dims,
            encoder=True
        )
        self.dims = dims

    def forward(self, x):
        out = self.layers(x)
        return ModelOutput(
            embedding=out[:, :self.dims[-1]],
            log_covariance=out[:, self.dims[-1]:]
            )
    
class Decoder(BaseDecoder):
    def __init__(self, dims):
        BaseDecoder.__init__(self)
        self.layers = make_linear_network(
            dims,
            encoder=False
        )
        self.dims = dims
        
    def forward(self, x):
        out = self.layers(x)
        return ModelOutput(
            reconstruction=out[:self.dims[-1]]
            )

config = VAE_IAF_Config(
    n_made_blocks=3,
    latent_dim=args.z_dim,
    input_dim=(26150,)
)

model = MultiVAEIAF(
    model_config=config,
    encoder=Encoder(dims=layers[::-1]),
    decoder=Decoder(dims=layers)
)


In [74]:
x = next(dataset.next_train_batch())
model(x, anneal=2)

ModelOutput([('loss', tensor(120.5386, grad_fn=<AddBackward0>)),
             ('elbo', tensor(76.3620, grad_fn=<AddBackward0>))])